In [23]:
import sys, os, warnings, geopandas as gpd, numpy as np, pandas as pd, folium
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
try:
    from shapely import make_valid     # Shapely 2.0+
    HAS_MAKE_VALID = True
except Exception:
    from shapely.validation import make_valid   # Shapely 1.8 fallback
    HAS_MAKE_VALID = False
from CompatnessClasses import CompactnessEngine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from factor_analyzer.rotator import Rotator
from sklearn.neighbors import NearestNeighbors
sys.path.append("../../../../Main/Erfan net crime/urban-network-analysis/src")
from street_network_processor import StreetNetworkProcessor
pd.set_option('display.float_format', '{:.4f}'.format)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
os.getcwd()

'C:\\Users\\erfan\\Dropbox (UNC Charlotte)\\Researches\\Erfan Dissertation\\Scripts\\py\\SUM_Charlotte'

In [24]:
#Importing datasets
# ABT = gpd.read_file("../../../../../Edited Subdivisions Zurikanen Erfan 4-20=2025/subs_plus_new/sub.gdb",layer='final_10_9_25')
# ABT["issue_date"] = ABT["issue_date"].dt.year.astype(str) # This is required for the folium cartography
ABT = gpd.read_file("../../../Data/Final_dataset/ABT/ABT.gpkg", layer="subdivisions")


# building_footprint_dataset = gpd.read_file('../../../Data/Final_dataset/ABT/outputs_building_overlap/Final_Buildings_SemanticIntegrated.gpkg', columns = ["geometry","HTD_SQ_FT"]).to_crs(ABT.crs)
# building_footprint_dataset = gpd.read_file('../../../Data/Final_dataset/ABT/outputs_building_overlap/Final_Buildings_SemanticIntegrated.gpkg').to_crs(ABT.crs)
# building_footprint_dataset['geometry'] = building_footprint_dataset.geometry.apply(make_valid)
# building_footprint_dataset_county = gpd.read_file("../../../Data/Original_dataset/Archive/Buildings_footprint_meck_01_24/Buildings.shp").to_crs(ABT.crs)
 
# tax_parcel_dataset = gpd.read_file("../../../Data/Original_dataset/Archive/ParcelZoningZipcode_meck_2023_08/ParcelZoningZipcode.shp", columns=["geometry","bldgtype"]).to_crs(ABT.crs)
# tax_parcel_cama_dataset = gpd.read_file("../../../Data/Original_dataset/original.gdb", layer = "Tax_CAMA_6_24", columns=["geometry","bldgtype"]).to_crs(ABT.crs)

# source_edges_path = "../../../../Main/Erfan net crime/data/Streets_Meckl_10_8_2023/processing_files/edges_version_2.shp"

# waterbody = gpd.read_file("../../../Data/Original_dataset/Archive/Ponds/Ponds.shp").to_crs(ABT.crs)

# street_network_df = gpd.read_file("../../../Data/Final_dataset/street_network_df.shp")

accessibility_ws = pd.read_csv("../../../Data/Final_dataset/ABT/WalkScore.csv")

In [ ]:
# Visualzing sample sub

sub_id_to_check = 4294  # change to one that exists


sub_row = ABT[ABT["subd_id"] == sub_id_to_check].iloc[0]
sub_geom = sub_row.geometry

# Find intersecting and clipped buildings
intersecting_buildings = building_footprint_dataset[building_footprint_dataset.intersects(sub_geom)].copy()
intersecting_buildings["geometry"] = intersecting_buildings.geometry.intersection(sub_geom)
intersecting_buildings = intersecting_buildings[~intersecting_buildings.geometry.is_empty]

# Convert to WGS84 for folium
sub_vis = gpd.GeoDataFrame([sub_row], crs=ABT.crs).to_crs(epsg=4326)
build_vis = intersecting_buildings.to_crs(epsg=4326)

# Initialize map
centroid = sub_vis.geometry.centroid.iloc[0]
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=16, tiles="CartoDB positron")

# Add subdivision boundary
folium.GeoJson(
    sub_vis,
    name="Subdivision",
    style_function=lambda f: {"color": "black", "weight": 2, "fill": False}
).add_to(m)

# Add buildings (clipped)
folium.GeoJson(
    build_vis,
    name="Buildings (clipped)",
    style_function=lambda f: {"color": "red", "weight": 1, "fillOpacity": 0.5},
).add_to(m)

folium.LayerControl().add_to(m)
m

# __1. Measuring the 1st indicator (Cohesion Index Intra-subdivision)__

**note: I ran the code in HPC environment**

In [ ]:
# ============================================================
#  STAGE 1: COMPUTE CONTENT-LEVEL COMPACTNESS METRICS
# ============================================================
content_cols = [
    "AI",        # Aggregation Index (higher = more compact)
    "PROX",      # Proximity Index   (higher = more compact)
    "ENN_MN",    # Nearest-Neighbor Distance (lower = more compact → inverted later)
    "ED",        # Edge Density      (lower = more compact → inverted later)
    "SHAPE_MN",
    "FRAC_MN"
]

# Initialize columns in ABT
for col in content_cols:
    ABT[col] = None

# Initialize Compactness Engine
compact_engine = CompactnessEngine(
    buildings_gdf=building_footprint_dataset.to_crs(ABT.crs)
)

# Helper function
def compute_compactness_for_row(sub_geom):
    try:
        metrics = compact_engine.compute_for_subdivision(
            subdivision=sub_geom
        )
        if metrics is None:
            return None
        return {col: metrics.get(col, None) for col in content_cols}
    except Exception as e:
        print("❌ Error:", e)
        return None

# Compute for all subdivisions
tqdm.pandas()

ABT["compactness_dict"] = ABT.geometry.progress_apply(
    compute_compactness_for_row
)

for col in content_cols:
    ABT[col] = ABT["compactness_dict"].apply(
        lambda d: d.get(col) if isinstance(d, dict) else None
    )

ABT.drop(columns=["compactness_dict"], inplace=True)
print("✅ Finished Stage 1: Extracted content-level compactness metrics.")

__ENTROPY__

In [46]:
# ============================================================
# Step 2A — REORIENTATION (ensure higher = more compact)
# ============================================================

ABT["ENN_inv"] = 1 / (ABT["ENN_MN"] + 1e-6)   # invert distance
ABT["ED_inv"]  = 1 / (ABT["ED"] + 1e-6)       # invert edge density
ABT["SHAPE_inv"] = 1 / (ABT["SHAPE_MN"] + 1e-6)  # invert shape mean
ABT["FRAC_inv"]  = 1 / (ABT["FRAC_MN"]  + 1e-6)  # invert fractal mean

# Define final metrics used in composite index
main_metrics = ["AI", "PROX", "ENN_inv", "ED_inv"]
optional_metrics = ["SHAPE_inv", "FRAC_inv"]

all_metrics = main_metrics + optional_metrics


# ============================================================
# Step 2B — NORMALIZATION (ONE TIME)
# ============================================================

def min_max_normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-9)

for m in all_metrics:
    ABT[m + "_norm"] = min_max_normalize(ABT[m])

print("✅ Finished Stage 2: Reoriented & normalized all metrics.")


# ============================================================
# Step 3A — ENTROPY WEIGHTS
# ============================================================

def compute_entropy_weights(df, metric_cols):
    X = df[metric_cols].fillna(0).values

    # Convert to proportions across subdivisions
    P = X / (X.sum(axis=0, keepdims=True) + 1e-12)

    # Compute entropy
    k = 1 / np.log(len(df))
    entropy = -k * np.sum(P * np.log(P + 1e-12), axis=0)

    # Compute redundancy
    d = 1 - entropy

    # Normalize to weights
    w = d / d.sum()

    return pd.Series(w, index=metric_cols)

entropy_weights = compute_entropy_weights(
    ABT, 
    [m + "_norm" for m in all_metrics]
)

print("\n📌 ENTROPY WEIGHTS:")
print(entropy_weights)
print("Sum of weights:", entropy_weights.sum())


# Final compactness score = weighted sum of normalized metrics

ABT["COMPACTNESS_SUM"] = 0

for m in all_metrics:
    ABT["COMPACTNESS_SUM"] += (
        ABT[m + "_norm"] * entropy_weights[m + "_norm"]
    )

print("✅ Stage 3 complete: Final COMPACTNESS_SUM index created.")

100%|██████████| 8376/8376 [05:23<00:00, 25.89it/s] 

✅ Finished Stage 1: Extracted content-level compactness metrics.
✅ Finished Stage 2: Reoriented & normalized all metrics.

📌 ENTROPY WEIGHTS:
AI_norm         0.0023
PROX_norm       0.1411
ENN_inv_norm    0.3642
ED_inv_norm     0.3049
SHAPE_MN_norm   0.1851
FRAC_MN_norm    0.0025
dtype: float64
Sum of weights: 1.0
✅ Stage 3 complete: Final COMPACTNESS_SUM index created.


# __2. Measuring the 2nd indicator (Inverse Distance to Nearest Human Activity Center Index)__ 

In [16]:
if "HAC_dist" not in ABT.columns:
    
    ABT["HAC_dist"] = None

    HAC_dataset = gpd.read_file("../../../Data/Original_dataset/original.gdb",layer='HAC')
    
    # mapping of HAC locations
    HAC_dataset_wgs = HAC_dataset.to_crs(epsg=4326)
    
    # Get the center of the map based on dataset bounds
    bounds = HAC_dataset_wgs.total_bounds  # [minx, miny, maxx, maxy]
    center = [(bounds[1] + bounds[3])/2, (bounds[0] + bounds[2])/2]
    
    # Create a Folium map
    m = folium.Map(location=center, zoom_start=12)
    
    # Add the HAC dataset
    folium.GeoJson(HAC_dataset_wgs).add_to(m);m
    
    nearest_distances = gpd.sjoin_nearest(
    ABT,
    HAC_dataset[['geometry']],   # only geometry, no extra columns
    how='left',
    distance_col='HAC_dist'
    )

    # Add distance in mile as a new column in the original GeoDataFrame
    ABT['HAC_dist'] = (nearest_distances['HAC_dist'] / 5280).round(2)
    
else:
    print("✅ HAC Index already exist).")

# __3. measuring the 3rd indicator (Cohesion Index Contextual)__

**note: I ran the code in HPC environment**


In [8]:
# ============================================================
# CONTEXTUAL COMPACTNESS AROUND SUBDIVISIONS (UPDATED & FINAL)
# ============================================================

# ---------------------------------------------------------
# 0. METRIC LISTS (ALL ALREADY DIRECTIONALLY ALIGNED)
# ---------------------------------------------------------
ctx_content_cols = [
    "AI",          # Aggregation Index (higher = more compact)
    "PROX",        # Proximity Index (higher = more compact)
    "ENN_inv",     # Inverted ENN_MN
    "ED_inv",      # Inverted Edge Density
    "SHAPE_inv",   # Inverted Mean Shape Index
    "FRAC_inv"     # Inverted Mean Fractal Dimension
]

# Initialize dictionary columns
ABT["ctx025_dict"] = None
ABT["ctx050_dict"] = None


# ---------------------------------------------------------
# 1. COMPACTNESS ENGINE (same as subdivision-level)
# ---------------------------------------------------------
compact_engine = CompactnessEngine(
    buildings_gdf=building_footprint_dataset.to_crs(ABT.crs)
)


# ---------------------------------------------------------
# 2. HELPER FUNCTION — CONTEXTUAL METRICS
# ---------------------------------------------------------
def compute_contextual_for_row(sub_geom, radius, resolution=5):
    """
    Compute contextual compactness metrics within a circular buffer
    centered on the subdivision centroid, excluding buildings inside
    the subdivision itself.
    """

    centroid = sub_geom.centroid
    buffer_geom = centroid.buffer(radius)

    # Spatial index query
    idx = list(building_footprint_dataset.sindex.intersection(buffer_geom.bounds))
    possible = building_footprint_dataset.iloc[idx]

    # Buildings inside buffer but outside the subdivision
    bld = possible[
        possible.geometry.intersects(buffer_geom)
        & ~possible.geometry.within(sub_geom)
    ]

    if len(bld) == 0:
        return None

    geoms = bld.geometry.tolist()

    # Rasterize buildings inside buffer
    raster = compact_engine._rasterize_buildings(
        geoms, buffer_geom, resolution
    )

    # ------------------ METRIC COMPUTATION ------------------

    # Aggregation Index
    AI = compact_engine.aggregation_index(raster)

    # Proximity Index (FRAGSTATS-style)
    PROX = compact_engine.proximity_index(
        geoms=geoms,
        subdivision=buffer_geom,
        resolution=resolution,
        search_radius=100
    )

    # Euclidean Nearest Neighbor (patch-based)
    ENN_MN = compact_engine.enn_mn_raster(
        raster=raster,
        subdivision=buffer_geom,
        resolution=resolution
    )
    ENN_inv = 1 / (ENN_MN + 1e-6)

    # Edge Density (class-level, m/ha)
    ED = compact_engine.edge_density_raster(
        raster=raster,
        subdivision=buffer_geom,
        resolution_feet=resolution
    )
    ED_inv = 1 / (ED + 1e-6)

    # Shape metrics (building geometry)
    SHAPE_MN = float(np.mean([
        compact_engine.building_shape_index(g) for g in geoms
    ]))
    FRAC_MN = float(np.nanmean([
        compact_engine.building_fractal_dimension(g) for g in geoms
    ]))

    SHAPE_inv = 1 / (SHAPE_MN + 1e-6)
    FRAC_inv  = 1 / (FRAC_MN  + 1e-6)

    return {
        "AI": AI,
        "PROX": PROX,
        "ENN_inv": ENN_inv,
        "ED_inv": ED_inv,
        "SHAPE_inv": SHAPE_inv,
        "FRAC_inv": FRAC_inv
    }


# ---------------------------------------------------------
# 3. APPLY TO ALL SUBDIVISIONS
# ---------------------------------------------------------
tqdm.pandas()

# 0.25-mile buffer (1320 ft)
ABT["ctx025_dict"] = ABT.geometry.progress_apply(
    lambda g: compute_contextual_for_row(g, radius=1320)
)

# 0.5-mile buffer (2640 ft)
ABT["ctx050_dict"] = ABT.geometry.progress_apply(
    lambda g: compute_contextual_for_row(g, radius=2640)
)

print("✅ Finished Stage 1: Extracted contextual raw metrics.")


# ---------------------------------------------------------
# 4. EXPAND DICTIONARIES INTO COLUMNS
# ---------------------------------------------------------
for col in ctx_content_cols:
    ABT[f"{col}_025"] = ABT["ctx025_dict"].apply(
        lambda d: d.get(col) if isinstance(d, dict) else None
    )
    ABT[f"{col}_050"] = ABT["ctx050_dict"].apply(
        lambda d: d.get(col) if isinstance(d, dict) else None
    )

ABT.drop(columns=["ctx025_dict", "ctx050_dict"], inplace=True)
print("🔍 Expanded contextual metrics into columns.")


# ---------------------------------------------------------
# 5. NORMALIZATION (MIN–MAX)
# ---------------------------------------------------------
def min_max_normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-9)

# Normalize 0.25-mile metrics
for m in ctx_content_cols:
    ABT[f"{m}_025_norm"] = min_max_normalize(ABT[f"{m}_025"])

# Normalize 0.5-mile metrics
for m in ctx_content_cols:
    ABT[f"{m}_050_norm"] = min_max_normalize(ABT[f"{m}_050"])

print("✅ Stage 2: Normalized contextual metrics.")


# ---------------------------------------------------------
# 6. ENTROPY WEIGHTING
# ---------------------------------------------------------
def compute_entropy_weights(df, metric_cols):
    X = df[metric_cols].fillna(0).values

    # Proportions
    P = X / (X.sum(axis=0, keepdims=True) + 1e-12)

    k = 1 / np.log(len(df))
    entropy = -k * np.sum(P * np.log(P + 1e-12), axis=0)

    redundancy = 1 - entropy
    weights = redundancy / redundancy.sum()

    return pd.Series(weights, index=metric_cols)


cols025 = [m + "_025_norm" for m in ctx_content_cols]
cols050 = [m + "_050_norm" for m in ctx_content_cols]

weights025 = compute_entropy_weights(ABT, cols025)
weights050 = compute_entropy_weights(ABT, cols050)

print("\n📌 ENTROPY WEIGHTS (0.25-mile):")
print(weights025)
print("\n📌 ENTROPY WEIGHTS (0.5-mile):")
print(weights050)


# ---------------------------------------------------------
# 7. FINAL CONTEXTUAL COMPACTNESS INDICES
# ---------------------------------------------------------
ABT["COMPACTNESS_SUM_ctx_025"] = 0.0
ABT["COMPACTNESS_SUM_ctx_050"] = 0.0

for m in ctx_content_cols:
    ABT["COMPACTNESS_SUM_ctx_025"] += (
        ABT[f"{m}_025_norm"] * weights025[f"{m}_025_norm"]
    )
    ABT["COMPACTNESS_SUM_ctx_050"] += (
        ABT[f"{m}_050_norm"] * weights050[f"{m}_050_norm"]
    )

print("🎉 SUCCESS: Created COMPACTNESS_SUM_ctx_025 and COMPACTNESS_SUM_ctx_050.")

 38%|███▊      | 3148/8376 [37:24<1:02:07,  1.40it/s]

KeyboardInterrupt



In [ ]:
    #Quality control

# --- Make sure both GeoDataFrames are in WGS84 (lat/lon) ---
subdivision_wgs = ABT.to_crs(4326)
buildings_wgs = building_footprint_dataset.to_crs(4326)

# --- Pick a sample subdivision by index ---
sample_idx = 42  # 👈 change this index to test another
sub = subdivision_wgs.loc[sample_idx]
centroid = sub.geometry.centroid

# --- Buffers (recreate in feet, then project to WGS84) ---
sub_proj = ABT.loc[[sample_idx]]  # use projected CRS for buffering
centroid_proj = sub_proj.geometry.centroid.iloc[0]
buffer_025_proj = centroid_proj.buffer(buffer_025_radius)
buffer_050_proj = centroid_proj.buffer(buffer_050_radius)

# Convert buffers to GeoDataFrames and reproject to WGS84
buffer_025 = gpd.GeoSeries([buffer_025_proj], crs=ABT.crs).to_crs(4326).iloc[0]
buffer_050 = gpd.GeoSeries([buffer_050_proj], crs=ABT.crs).to_crs(4326).iloc[0]

# --- Buildings intersecting buffers (still use projected CRS for accuracy) ---
bldgs_025_proj = building_footprint_dataset[
    building_footprint_dataset.geometry.intersects(buffer_025_proj) &
    ~building_footprint_dataset.geometry.within(sub_proj.geometry.iloc[0])
]
bldgs_050_proj = building_footprint_dataset[
    building_footprint_dataset.geometry.intersects(buffer_050_proj) &
    ~building_footprint_dataset.geometry.within(sub_proj.geometry.iloc[0])
]

# Reproject selected buildings to WGS84 for Folium
bldgs_025 = bldgs_025_proj.to_crs(4326)
bldgs_050 = bldgs_050_proj.to_crs(4326)

# --- Create Folium map centered on the subdivision centroid ---
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=16, tiles="CartoDB positron")

# Subdivision polygon
folium.GeoJson(sub.geometry, name="Subdivision",
               style_function=lambda x: {"color": "blue", "weight": 3, "fill": False}).add_to(m)

# Buffers
folium.GeoJson(buffer_025, name="0.25 mi Buffer",
               style_function=lambda x: {"color": "orange", "weight": 2, "fill": False}).add_to(m)
folium.GeoJson(buffer_050, name="0.5 mi Buffer",
               style_function=lambda x: {"color": "red", "weight": 2, "fill": False}).add_to(m)

# Buildings
folium.GeoJson(bldgs_025, name="Buildings (0.25 mi)",
               style_function=lambda x: {"color": "orange", "weight": 1}).add_to(m)
folium.GeoJson(bldgs_050, name="Buildings (0.5 mi)",
               style_function=lambda x: {"color": "red", "weight": 1}).add_to(m)

# Centroid marker
folium.Marker(
    location=[centroid.y, centroid.x],
    popup=(f"<b>Subdivision {sample_idx}</b><br>"
           f"CI (0.25 mi): {ABT.loc[sample_idx, 'ci_025mi']}<br>"
           f"CI (0.5 mi): {ABT.loc[sample_idx, 'ci_050mi']}"),
    icon=folium.Icon(color="blue", icon="home")
).add_to(m)

# Layer control
folium.LayerControl().add_to(m)

m

# __4. Measuring the 4th indicator (BAD)__

In [ ]:
if 'BAD' in ABT.columns:
    
    ABT['BAD']= None

    # --- 2) Fix invalid geometries ---
    if HAS_MAKE_VALID:
        ABT['geometry'] = ABT.geometry.map(make_valid)
        building_footprint_dataset['geometry'] = building_footprint_dataset.geometry.map(make_valid)
    else:
        ABT['geometry'] = ABT.buffer(0)
        building_footprint_dataset['geometry'] = building_footprint_dataset.buffer(0)
    
    # --- 3) Compute subdivision land area ---
    ABT['land_area'] = ABT.geometry.area
    
    # --- 4) Overlay in chunks to track progress ---
    subs = ABT[['subd_id', 'geometry']]
    blds = building_footprint_dataset[['geometry']]
    
    chunk_size = 5000  # adjust depending on your RAM
    intersections_list = []
    
    print("Performing overlay in chunks...")
    for i in tqdm(range(0, len(blds), chunk_size), desc="Overlay progress"):
        chunk = blds.iloc[i:i+chunk_size]
        inter = gpd.overlay(chunk, subs, how='intersection', keep_geom_type=True)
        intersections_list.append(inter)
    
    intersections = pd.concat(intersections_list, ignore_index=True)
    
    # --- 5) Sum intersected areas per subdivision ---
    print("Computing built-up area sums...")
    intersections['built_piece_area'] = intersections.geometry.area
    built_sum = (intersections
                 .groupby('subd_id', as_index=False)['built_piece_area']
                 .sum()
                 .rename(columns={'built_piece_area': 'built_area'}))
    
    # --- 6) Merge back and compute BAD ---
    print("Final merge and ratio computation...")
    ABT = ABT.merge(built_sum, on='subd_id', how='left')
    ABT['built_area'] = ABT['built_area'].fillna(0.0)
    ABT['BAD'] = (ABT['built_area'] / ABT['land_area']).replace([float('inf')], 0).round(3)
else:
    print("✅ BAD Index already exist).")

# __5. Measuring the 5th indicator (FAR)__

In [22]:
if 'FAR'  in ABT.columns:
    
    ABT["FAR"] = None
    
    # --------------------------------------------------------------
    # 1. CREATE INDEX-BASED UNIQUE BUILDING ID
    # --------------------------------------------------------------
    
    print("Assigning index-based building IDs...")
    
    building_footprint_dataset = (
        building_footprint_dataset.reset_index()
        .rename(columns={"index": "bldg_idx"})
    )
    
    # --------------------------------------------------------------
    # 2. RUN SPATIAL OVERLAY / INTERSECTION
    # --------------------------------------------------------------
    
    print("Running spatial overlay/intersection (this may take time)...")
    
    intersections = gpd.overlay(
        building_footprint_dataset[["bldg_idx", "geometry"]],
        ABT[["subd_id", "geometry"]],
        how="intersection"
    )
    
    print(f"Overlay complete. {len(intersections)} intersecting building pieces.\n")
    
    # --------------------------------------------------------------
    # 3. COMPUTE AREAS
    # --------------------------------------------------------------
    
    print("Computing intersection and building footprint areas...")
    
    intersections["INTERSECT_AREA"] = intersections.geometry.area
    building_footprint_dataset["BLDG_AREA"] = building_footprint_dataset.geometry.area
    
    print("Areas computed.\n")
    
    # --------------------------------------------------------------
    # 4. MERGE BUILDING ATTRIBUTES BACK INTO INTERSECTIONS
    # --------------------------------------------------------------
    
    print("Merging building attributes back into intersections...")
    
    intersections = intersections.merge(
        building_footprint_dataset[["bldg_idx", "BLDG_AREA", "HTD_SQ_FT"]],
        on="bldg_idx",
        how="left"
    )
    
    print("Attributes merged.\n")
    
    # --------------------------------------------------------------
    # 5. CALCULATE PROPORTIONAL HEATED SQ FT
    # --------------------------------------------------------------
    
    print("Calculating proportional heated square feet...")
    
    intersections["AREA_PROPORTION"] = (
        intersections["INTERSECT_AREA"] / intersections["BLDG_AREA"]
    ).clip(upper=1)
    
    intersections["HTD_SQ_FT_PROPORTIONAL"] = (
        intersections["HTD_SQ_FT"] * intersections["AREA_PROPORTION"]
    )
    
    print("Proportional HTD_SQ_FT computed.\n")
    
    # --------------------------------------------------------------
    # 6. AGGREGATE BY PARCEL (subd_id)
    # --------------------------------------------------------------
    
    print("Aggregating proportional heated sqft by subd_id...")
    
    htd_sum = (
        intersections
        .groupby("subd_id")["HTD_SQ_FT_PROPORTIONAL"]
        .sum()
        .reset_index()
        .rename(columns={"HTD_SQ_FT_PROPORTIONAL": "TOTAL_HTD_SQ_FT"})
    )
    
    print("Aggregation complete.\n")
    
    # --------------------------------------------------------------
    # 7. COMPUTE PARCEL AREA
    # --------------------------------------------------------------
    
    print("Computing parcel areas...")
    
    ABT["PARCEL_AREA_SQFT"] = ABT.geometry.area
    
    # --------------------------------------------------------------
    # 8. MERGE HEATED AREA BACK INTO ABT
    # --------------------------------------------------------------
    
    print("Merging heated sqft into ABT...")
    
    ABT = ABT.merge(htd_sum, on="subd_id", how="left")
    ABT["TOTAL_HTD_SQ_FT"] = ABT["TOTAL_HTD_SQ_FT"].fillna(0)
    
    print("Merge complete.\n")
    
    # --------------------------------------------------------------
    # 9. COMPUTE FAR
    # --------------------------------------------------------------
    
    print("Computing FAR...")
    
    ABT["FAR"] = ABT["TOTAL_HTD_SQ_FT"] / ABT["PARCEL_AREA_SQFT"]
    
    print("FAR calculation completed.\n")

Assigning index-based building IDs...
Running spatial overlay/intersection (this may take time)...
Overlay complete. 240687 intersecting building pieces.

Computing intersection and building footprint areas...
Areas computed.

Merging building attributes back into intersections...
Attributes merged.

Calculating proportional heated square feet...
Proportional HTD_SQ_FT computed.

Aggregating proportional heated sqft by subd_id...
Aggregation complete.

Computing parcel areas...
Merging heated sqft into ABT...
Merge complete.

Computing FAR...
FAR calculation completed.



# __6. Measuring the 6th indicator (SHD)__


In [29]:
if 'SHD' in ABT.columns:

    ABT['SHD'] = None
    category_map = {
        'SINGLE FAMILY RESIDENTIAL': 'Residential - Single Family',
        'SINGLE FAMILY RES TINY HOME': 'Residential - Single Family',
        'SINGLE FAMILY MODULAR': 'Residential - Single Family',
        'SINGLE FAMILY HISTORICAL PROPERTY': 'Residential - Single Family',
        'DUPLEX-TRIPLEX': 'Residential - Multi Family',
        'TOWNHOUSE': 'Residential - Multi Family',
        'APARTMENT-GARDEN <=3 STORIES': 'Residential - Multi Family',
        'RESIDENTIAL CONDOMINIUM < 7 STORIES': 'Residential - Multi Family',
        'MID RISE APARTMENT 4-6 STORIES': 'Residential - Multi Family',
        'RESIDENTIAL CONDO HI-RISE > 6 STORIES': 'Residential - Multi Family',
        'APARTMENT-HIRISE >=7 STORIES': 'Residential - Multi Family',
        'APARTMENT-TOWNHOUSE': 'Residential - Multi Family',
        'PATIO HOME': 'Residential - Single Family',
        'GROUP HOME': 'Residential - Multi Family',
        'DORMITORY': 'Residential - Multi Family',
        'OFFICE/RETAIL/MULTI-FAMILY': 'Mixed Use / Common Area',
        'CLUB HOUSE HOA NO VALUE': 'Mixed Use / Common Area',
        'NO VALUE IMPROVMENT': 'Mixed Use / Common Area',
        'CLUB-LODGE': 'Mixed Use / Common Area',
        'RECREATIONAL CENTER': 'Mixed Use / Common Area',
        'RETAIL': 'Commercial / Retail',
        'RETAIL CONDO': 'Commercial / Retail',
        'RETAIL/CONVENIENCE STORE': 'Commercial / Retail',
        'FURNITURE SHOWROOM': 'Commercial / Retail',
        'DEALERSHIP SHOWROOM': 'Commercial / Retail',
        'CONVENIENCE STORE': 'Commercial / Retail',
        'DISCOUNT/DEPARTMENT STORE': 'Commercial / Retail',
        'DEPARTMENT STORE': 'Commercial / Retail',
        'MINI MART CONVENIENCE STORE': 'Commercial / Retail',
        'SUPERMARKET': 'Commercial / Retail',
        'STRIP SHOPPING CENTER': 'Commercial / Retail',
        'SHOPPING MALL': 'Commercial / Retail',
        'FAST FOOD': 'Commercial / Retail',
        'FAST FOOD/CONVENIENCE STORE': 'Commercial / Retail',
        'RESTAURANT': 'Commercial / Retail',
        'NIGHTCLUB/LOUNGE': 'Commercial / Retail',
        'DAYCARE': 'Commercial / Retail',
        'BANK': 'Commercial / Retail',
        'THEATRE': 'Commercial / Retail',
        'FITNESS CENTER/HEALTH CLUB': 'Commercial / Retail',
        'ANIMAL DAY SPA': 'Commercial / Retail',
        'CAR WASH FULL-SERVICE TYPE': 'Commercial / Retail',
        'CAR WASH SELF-SERVICE TYPE': 'Commercial / Retail',
        'CAR WASH DRIVE-THRU TYPE': 'Commercial / Retail',
        'MINI SPECIALTY AUTOMOTIVE': 'Commercial / Retail',
        'AUTO SERVICE CENTER': 'Commercial / Retail',
        'SERVICE GARAGE': 'Commercial / Retail',
        'OFFICE A CLASS': 'Office',
        'OFFICE B CLASS': 'Office',
        'OFFICE C CLASS': 'Office',
        'OFFICE HI-RISE A > 4 STORIES': 'Office',
        'OFFICE HI-RISE B > 4 STORIES': 'Office',
        'OFFICE MODULAR': 'Office',
        'OFFICE CONDOMINIUM': 'Office',
        'SFR TO OFFICE': 'Office',
        'OFFICE & RETAIL': 'Office',
        'LAB- RESEARCH': 'Office',
        'MEDICAL/DENTAL': 'Medical',
        'MEDICAL CONDOMINIUM': 'Medical',
        'HOSPITAL-PRIVATE': 'Medical',
        'HOSPITAL-PUBLIC': 'Medical',
        'NURSING HOME/CONVALESCENT HOSPITAL': 'Medical',
        'URGENT CARE FACILITY': 'Medical',
        'VETERINARIAN OFFICE': 'Medical',
        'ASSISTED LIVING': 'Medical',
        'HOME FOR THE ELDERLY': 'Medical',
        'HOTEL FULL SERVICE': 'Hospitality',
        'HOTEL LIMITED SERVICE': 'Hospitality',
        'HOTEL/EXTENDED STAY': 'Hospitality',
        'MOTEL': 'Hospitality',
        'MOTEL/HOTEL LODGING < 7 STORIES': 'Hospitality',
        'HOTEL LODGING HI-RISE > 6 STORIES': 'Hospitality',
        'BED & BREAKFEST': 'Hospitality',
        'MICRO BREWERY/WINERY': 'Hospitality',
        'WAREHOUSE': 'Industrial / Warehousing',
        'MEGA WAREHOUSE': 'Industrial / Warehousing',
        'MINI WAREHOUSE': 'Industrial / Warehousing',
        'MINI WAREHOUSE CLIMATE CONTROL': 'Industrial / Warehousing',
        'WAREHOUSE CONDOMINIUM': 'Industrial / Warehousing',
        'WAREHOUSE DISTRIBUTION': 'Industrial / Warehousing',
        'WAREHOUSE DISCOUNT STORE': 'Industrial / Warehousing',
        'TRANSIT/TRUCK WAREHOUSE': 'Industrial / Warehousing',
        'INDUSTRIAL': 'Industrial / Warehousing',
        'INDUSTRIAL FLEX': 'Industrial / Warehousing',
        'INDUSTRIAL RESEARCH & DEVELOPMENT': 'Industrial / Warehousing',
        'LIGHT MANUFACTURING': 'Industrial / Warehousing',
        'LIGHT MANUFACTURING > 75,000 SF': 'Industrial / Warehousing',
        'HEAVY MANUFACTURING': 'Industrial / Warehousing',
        'PREFAB WAREHOUSE': 'Industrial / Warehousing',
        'BOTTLING PLANT': 'Industrial / Warehousing',
        'PACKING PLANT': 'Industrial / Warehousing',
        'COMPUTER DATA CENTER': 'Industrial / Warehousing',
        'COLD STORAGE': 'Industrial / Warehousing',
        'LUMBER YARD': 'Industrial / Warehousing',
        'CHURCH': 'Religious / Church',
        'SCHOOL-PUBLIC': 'Other / Infrastructure / Utilities',
        'SCHOOL-PRIVATE': 'Other / Infrastructure / Utilities',
        'COLLEGE/UNIVERSITY': 'Other / Infrastructure / Utilities',
        'LIBRARY': 'Other / Infrastructure / Utilities',
        'MUNICIPAL': 'Other / Infrastructure / Utilities',
        'COUNTY': 'Other / Infrastructure / Utilities',
        'STATE': 'Other / Infrastructure / Utilities',
        'FEDERAL': 'Other / Infrastructure / Utilities',
        'UTILITY/MECHANICAL EQUIPMENT BUILDING': 'Other / Infrastructure / Utilities',
        'CELLULAR EQUIPMENT SHED - STATE CERTIFIED': 'Other / Infrastructure / Utilities',
        'Passenger Terminal': 'Other / Infrastructure / Utilities',
        'CONVENTION CENTER': 'Other / Infrastructure / Utilities',
        'PARKING GARAGE': 'Other / Infrastructure / Utilities',
        'UNDERGROUND PARKING': 'Other / Infrastructure / Utilities',
        'STABLE': 'Other / Infrastructure / Utilities',
        'COUNTRY CLUB': 'Other / Infrastructure / Utilities',
        'STADIUM': 'Other / Infrastructure / Utilities',
        'ARENA': 'Other / Infrastructure / Utilities',
        'SPORTS ENTERTAINMENT CENTER': 'Other / Infrastructure / Utilities',
        'MANUFACTURED HOME-DOUBLEWIDE': 'Residential - Single Family',
        'MANUFACTURED HOME-SINGLEWIDE': 'Residential - Single Family',
        'FUNERAL HOME/MORTUARY': 'Medical',
        'DRUG STORE': 'Commercial / Retail',
        'COMMERCIAL CONDOMINIUM': 'Commercial / Retail',
        'MOTOR SPORTS GARAGE': 'Commercial / Retail',
        'BOWLING ALLEY/SKATING RINK': 'Commercial / Retail',
        'CONTINUING CARE': 'Medical',
    } # Remapping the building types
    tax_parcel_cama_dataset['bldgtype_grouped'] = tax_parcel_cama_dataset['bldgtype'].map(category_map)
    
    for idx, row in tqdm(ABT.iterrows(), total=len(ABT)):
        poly = row.geometry
        land_in_sub = tax_parcel_cama_dataset[tax_parcel_cama_dataset.geometry.intersects(poly)].copy()
        
        # Clip land use polygons to the subdivision
        land_in_sub['geometry'] = land_in_sub.geometry.intersection(poly)
        land_in_sub = land_in_sub[~land_in_sub.is_empty]
        
        if land_in_sub.empty:
            ABT.at[idx, 'SHD'] = None
            continue
    
        # Calculate area
        land_in_sub['area'] = land_in_sub.geometry.area
    
        # Group by land use type and sum area
        grouped = land_in_sub.groupby('bldgtype')['area'].sum()  #'bldgtype' column
        total_area = grouped.sum()
        proportions = grouped / total_area
    
        # Shannon entropy
        S = len(proportions)
        if S > 1:
            shd = -np.sum(proportions * np.log(proportions)) / np.log(S)
        else:
            shd = 0  # Only one land use → no diversity
    
        ABT.at[idx, 'SHD'] = shd
        ABT['SHD'] = pd.to_numeric(ABT['SHD'], errors='coerce').round(2)
        
else:
    print("✅ SHD Index already exist).")


100%|██████████| 6634/6634 [03:02<00:00, 36.42it/s]


# __7. Measuring the 7th indicator (RJD) and the 8th indicator (AND)__

In [14]:
network_measures = ['int_den0_1', 'nd_deg02_1', 'int_den0_2', 'nd_deg05_y', 'int_den0_3', 'nd_deg07_1', 'int_den1_1', 'nd_deg10_y']
if not all(col in ABT.columns for col in network_measures):
    ABT = pd.merge(ABT,street_network_df[['subd_id'] + network_measures],on="subd_id", how="left")
else:
    print("✅ RJD and AND Index already exist.")

# __9. Measuring the 9th indicator (APT) and the 10th indicator (AUO)__

In [36]:
accessibility_measures = ["groceries_ws", "transit_ws"]
if not all(col in ABT.columns for col in accessibility_measures):
    ABT = pd.merge(ABT,accessibility_ws[['subd_id'] + accessibility_measures],on="subd_id", how="left")
else:
    print("✅ APT and AUO Index already exist.")

In [ ]:
# accessibility_measures = ["groceries_ws", "transit_ws"]
# missing_code = 999  # value representing missing
# if not all(col in ABT.columns for col in accessibility_measures):
#     ABT = pd.merge(ABT,accessibility_ws[['subd_id'] + accessibility_measures], on="subd_id", how="left")
# else:
#     print("✅ APT and AUO Index already exist.")
# 
# 
# # Handling missing values
# 
# # ----------------------------------------------
# # STEP 1 — Ensure required columns exist
# # ----------------------------------------------
# print("🔍 Checking for required accessibility columns in ABT...")
# 
# missing_cols = [col for col in accessibility_measures if col not in ABT.columns]
# 
# if missing_cols:
#     print(f"⚠️ Missing columns detected in ABT: {missing_cols}")
#     print("➡️ Attempting to merge them from accessibility_ws...")
# 
#     ABT = pd.merge(
#         ABT,
#         accessibility_ws[["subd_id"] + missing_cols],
#         on="subd_id",
#         how="left"
#     )
# 
#     print(f"✔ Successfully merged missing columns: {missing_cols}")
# else:
#     print("✅ All accessibility columns already exist in ABT.")
# 
# 
# # ----------------------------------------------
# # STEP 2 — Compute centroid coordinates
# # ----------------------------------------------
# print("\n📌 Extracting centroid coordinates from ABT geometry...")
# 
# if ABT.geometry.is_empty.any():
#     print("⚠️ Warning: Some geometries are empty. NN matching may fail for those rows.")
# 
# # Compute centroids safely
# ABT["cx"] = ABT.geometry.centroid.x
# ABT["cy"] = ABT.geometry.centroid.y
# 
# print("✔ Centroid coordinates computed: using columns 'cx' and 'cy'.")
# 
# 
# # ----------------------------------------------
# # STEP 3 — Spatial nearest-neighbor imputation
# # ----------------------------------------------
# print("\n🔍 Beginning spatial nearest-neighbor imputation for 999-coded missing values...\n")
# 
# df = ABT.copy()  # Safe working copy
# 
# for col in accessibility_measures:
#     print(f"--- Processing column: '{col}' ---")
# 
#     if col not in df.columns:
#         print(f"❌ Column '{col}' not found even after merge. Skipping.")
#         continue
# 
#     # Identify missing vs. valid rows
#     mask_missing = df[col] == missing_code
#     mask_valid = ~mask_missing
# 
#     num_missing = mask_missing.sum()
#     num_valid = mask_valid.sum()
# 
#     print(f"• Valid rows: {num_valid}")
#     print(f"• Missing (999) rows: {num_missing}")
# 
#     if num_missing == 0:
#         print(f"✔ No 999-coded values in '{col}'. Moving on.\n")
#         continue
# 
#     # Coordinates
#     coords_valid = df.loc[mask_valid, ["cx", "cy"]].values
#     coords_missing = df.loc[mask_missing, ["cx", "cy"]].values
# 
#     print("  → Fitting NearestNeighbors model on valid spatial points...")
#     nn = NearestNeighbors(n_neighbors=1)
#     nn.fit(coords_valid)
# 
#     print("  → Finding nearest valid neighbors...")
#     distances, indices = nn.kneighbors(coords_missing)
# 
#     # Extract nearest values
#     nearest_values = df.loc[mask_valid, col].iloc[indices.flatten()].values
# 
#     # Impute
#     df.loc[mask_missing, col] = nearest_values
# 
#     print(f"✔ Imputed {num_missing} missing (999) values in '{col}' using nearest centroid.\n")
# 
# 
# # ----------------------------------------------
# # STEP 4 — Finalize output
# # ----------------------------------------------
# ABT = df.drop(columns=["cx", "cy"])  # Remove helper columns
# print("🏁 Spatial NN imputation complete. ABT updated successfully.")

In [32]:
ABT = ABT.drop(['groceries_ws_x', 'transit_ws_x'], axis=1)

In [19]:
ABT = ABT.rename(columns={
    'int_den0_1': 'int_den025',
    'nd_deg02_1': 'nd_deg025',
    'int_den0_2': 'int_den05',
    'nd_deg05_y': 'nd_deg05',
    'int_den0_3': 'int_den075',
    'nd_deg07_1': 'nd_deg075',
    'int_den1_1': 'int_den1',
    'nd_deg10_y': 'nd_deg1'
})

In [33]:
ABT.columns

Index(['subd_id', 'issue_date', 'year', 'HAC_dist', 'BAD', 'FAR', 'SHD',
       'area_acre', 'AI', 'PROX', 'ENN_MN', 'ED', 'SHAPE_MN', 'FRAC_MN',
       'ENN_inv', 'ED_inv', 'SHAPE_inv', 'FRAC_inv', 'AI_norm', 'PROX_norm',
       'ENN_inv_norm', 'ED_inv_norm', 'SHAPE_inv_norm', 'FRAC_inv_norm',
       'COMPACTNESS_SUM', 'int_den025', 'nd_deg025', 'int_den05', 'nd_deg05',
       'int_den075', 'nd_deg075', 'int_den1', 'nd_deg1', 'geometry'],
      dtype='object')

In [38]:
ABT.to_file("../../../Data/Final_dataset/ABT/ABT.gpkg", layer="subdivisions")

# __11. Measuring the 11th indicator (Green Coverage)__
